In [2]:
import cosmofast as cf
from astropy.io import fits
import numpy as np
foo = fits.open("cosmofast/planck_18/data/legacy-commander/sigma.fits")

In [4]:
foo.info()

Filename: cosmofast/planck_2018/data/commander/sigma.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      19   (1000, 249, 3)   float64   
  1                1 ImageHDU         6   (249,)   float64   
  2                1 ImageHDU         7   (249, 249)   float64   
  3                1 ImageHDU         7   (249,)   float64   


In [5]:
test = np.load("cosmofast/tests/test_commander.npz")
test['test_param'].shape

(31,)

In [8]:
cl2x = np.ascontiguousarray(
    np.moveaxis(foo[0].data.T[:, :28, :], 0, 2)).byteswap().newbyteorder()
mu = foo[1].data[:28].byteswap().newbyteorder()
cov = foo[2].data[:28, :28].byteswap().newbyteorder()
mu_offset = foo[3].data[:28].byteswap().newbyteorder()
cov_inv = np.linalg.inv(cov)

In [13]:
offset = np.empty(1)
cf.planck_18._commander._commander_f(
    mu_offset * 2 * np.pi / np.arange(2, 30) / (np.arange(2, 30) + 1), 1.0,
    offset, cl2x, mu, cov_inv)
offset = float(offset)
offset

-155.17039124516077

In [14]:
test_value = np.empty(1)
cf.planck_18._commander._commander_f(
    test['test_param'][2:30], test['test_param'][30], test_value, cl2x, mu, cov_inv)
test_value - offset, test['test_value']

(array([-11.5889457]), array([-11.58894468]))

In [19]:
np.savez_compressed('cosmofast/planck_18/data/commander.npz',
                    cl2x=cl2x, mu=mu, cov=cov, offset=offset)

In [1]:
import cosmofast as cf
import numpy as npimport cosmofast as cf
import numpy as np
foo = np.load('cosmofast/planck_18/data/commander.npz')
test = np.load("cosmofast/tests/test_commander.npz")
foo = np.load('cosmofast/planck_18/data/commander.npz')
test = np.load("cosmofast/tests/test_commander.npz")

In [2]:
cl2x = foo['cl2x']
mu = foo['mu']
cov = foo['cov']
cov_inv = np.linalg.inv(cov)
offset = foo['offset']

In [3]:
test_value = np.empty(1)
cf.planck_18._commander._commander_f(
    test['test_param'][2:30], test['test_param'][30], test_value, cl2x, mu, cov_inv)
test_value - offset

array([-11.5889457])